In [34]:
import torch
import json
import numpy as np
import math
import cv2

In [35]:
# cuda 연결 확인 코드
import torch
print(torch.cuda.is_available())

True


In [36]:
def cal_2d_distance(x1, y1, x2, y2):
    x = x2 - x1
    y = y2 - y1
    return math.sqrt(x**2 + y**2)

def cal_3d_distance(p1, p2):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    
    x = x2 - x1
    y = y2 - y1
    z = z2 - z1
    return math.sqrt(x**2 + y**2 + z**2)

In [37]:
file_path = 'AlphaPose/examples/results/alphapose_unnormal.json'

with open(file_path, "r") as f:
    alpha_data = json.load(f)

print("data type:", type(alpha_data))
print("data len:", len(alpha_data))
print("data[0] type:", type(alpha_data[0]))
print("data[0] keys:", list(alpha_data[0].keys()))
print("data[0]['keypoints'] type:", type(alpha_data[0]['keypoints']))
print("data[0]['keypoints'] len:", len(alpha_data[0]['keypoints']))

data type: <class 'list'>
data len: 129
data[0] type: <class 'dict'>
data[0] keys: ['image_id', 'category_id', 'keypoints', 'score', 'box', 'idx']
data[0]['keypoints'] type: <class 'list'>
data[0]['keypoints'] len: 78


In [38]:
k = ['Nose', 'LEye', 'REye', 'LEar', 'REar', 'LShoulder', 'RShoulder', 'LElbow', 'RElbow', 'LWrist', 'RWrist',
     'LHip', 'RHip', 'LKnee', 'Rknee', 'LAnkle', 'RAnkle', 'Head', 'Neck', 'Hip', 'LBigToe', 'RBigToe', 'LSmallToe', 'RSmallToe', 'LHeel', 'RHeel']
ex = alpha_data[10]['keypoints']
print("0.jpg value------------------------------------------------------------")
for i in range(26*3):
    if i % 3 == 0: print(k[i//3], [i, i+2], ":", ex[i+0:i+3])

0.jpg value------------------------------------------------------------
Nose [0, 2] : [388.397705078125, 123.46035766601562, 0.9358507394790649]
LEye [3, 5] : [411.47442626953125, 100.38365936279297, 0.9522897601127625]
REye [6, 8] : [365.3210144042969, 88.84530639648438, 0.9155483245849609]
LEar [9, 11] : [446.0894775390625, 123.46035766601562, 0.9784330725669861]
REar [12, 14] : [330.7059631347656, 123.46035766601562, 0.9481452107429504]
LShoulder [15, 17] : [515.319580078125, 238.84385681152344, 0.9357389807701111]
RShoulder [18, 20] : [284.55255126953125, 296.5356140136719, 0.9144362807273865]
LElbow [21, 23] : [549.9346313476562, 423.45745849609375, 0.8902885317802429]
RElbow [24, 26] : [284.55255126953125, 481.14923095703125, 0.9281059503555298]
LWrist [27, 29] : [549.9346313476562, 573.4560546875, 0.9491968154907227]
RWrist [30, 32] : [284.55255126953125, 642.6861572265625, 0.9031567573547363]
LHip [33, 35] : [480.70452880859375, 608.071044921875, 0.9180561900138855]
RHip [36, 3

In [39]:
# 골반 기울기 검사
print("\n-------------골반 기울기-------------")
fframe = 0 # 전면 이미지 or 프레임
val = alpha_data[fframe]['keypoints']
l_hip = val[33:35]
r_hip = val[36:38]
hip_rad = math.atan2(l_hip[1] - r_hip[1], l_hip[0] - r_hip[0])
hip_deg = (hip_rad*180)/math.pi
print("rad:", hip_rad, "/ abs rad:", abs(hip_rad))
print("deg:", hip_deg, "/ abs_deg:", abs(hip_deg))

# 어깨 기울기 검사
print("\n-------------어깨 기울기-------------")
fframe = 10 # 전면 이미지 or 프레임
val = alpha_data[fframe]['keypoints']
l_shoulder = val[15:17]
r_shoulder = val[18:20]
shoulder_rad = math.atan2(l_shoulder[1] - r_shoulder[1], l_shoulder[0] - r_shoulder[0])
shoulder_deg = (shoulder_rad*180)/math.pi
print("rad:", shoulder_rad, "/ abs rad:", abs(shoulder_rad))
print("deg:", shoulder_deg, "/ abs_deg:", abs(shoulder_deg))


-------------골반 기울기-------------
rad: -0.09066019046792476 / abs rad: 0.09066019046792476
deg: -5.194446283664264 / abs_deg: 5.194446283664264

-------------어깨 기울기-------------
rad: -0.24497866312686414 / abs rad: 0.24497866312686414
deg: -14.036243467926477 / abs_deg: 14.036243467926477


In [40]:
file_path = 'MotionBERT/examples/results/motionbert_unnormal.npy'
motion_data = np.load(file_path)
print("data type:", type(motion_data))
print("data shape:", motion_data.shape) # (프레임 개수, 키포인트, (x,y,z) 좌표) 

data type: <class 'numpy.ndarray'>
data shape: (129, 17, 3)


In [41]:
k = ['root', 'RHip', 'RKnee', 'RAnkle', 'LHip', 'LKnee', 'LAnkle', 'torso', 'neck', 'nose', 'head', 'LShoulder', 'LElbow', 'LWrist', 'RShoulder', 'RElbow', 'RWrist']
frame = 0
print(frame, "frame---------------------------------------")
for i, key in enumerate(motion_data[frame]):
    print(i, "-", k[i], ":", key)

0 frame---------------------------------------
0 - root : [0.03768173 0.05005858 0.        ]
1 - RHip : [-0.1000087   0.08143686  0.00129454]
2 - RKnee : [-0.08476543  0.5595484   0.11303172]
3 - RAnkle : [-0.09197865  0.9423065   0.23928683]
4 - LHip : [0.16553575 0.06564985 0.03721961]
5 - LKnee : [0.1290741  0.5565841  0.12816444]
6 - LAnkle : [0.10348095 0.94157517 0.24331982]
7 - torso : [ 0.014557   -0.30375177 -0.05320794]
8 - neck : [-0.02445125 -0.645584   -0.14199507]
9 - nose : [-0.00713882 -0.8157228  -0.26700154]
10 - head : [-0.01392266 -0.9944185  -0.24872652]
11 - LShoulder : [ 0.21354595 -0.6147522  -0.10536033]
12 - LElbow : [ 0.27860343 -0.27429682  0.06236859]
13 - LWrist : [0.27588317 0.01656048 0.07011461]
14 - RShoulder : [-0.21510932 -0.49337065 -0.11883163]
15 - RElbow : [-0.20651954 -0.1483562   0.03170614]
16 - RWrist : [-0.21250343  0.14715345  0.09092017]


In [42]:
# 거북목 증후군 검사
print("\n---------------거북목---------------")
frame = 10 # 우측면 이미지 or 프레임
val = motion_data[frame]
head = val[10]
neck = val[8]
tutle_xz = cal_2d_distance(head[0], head[2], neck[0], neck[2])
turtle_rad_3d = math.atan2(head[1] - neck[1], tutle_xz)
turtle_deg_3d = (turtle_rad_3d*180)/math.pi
print("rad:", turtle_rad_3d, "/ abs rad:", abs(turtle_rad_3d))
print("deg:", turtle_deg_3d, "/ abs_deg:", abs(turtle_deg_3d))

# 골반 기울기 검사
print("\n-------------골반 기울기-------------")
fframe = 0 # 전면 이미지 or 프레임
val = motion_data[fframe]
l_hip = val[4]
r_hip = val[1]
hip_xz = cal_2d_distance(l_hip[0], l_hip[2], r_hip[0], r_hip[2])
hip_rad_3d = math.atan2(l_hip[1] - r_hip[1], hip_xz)
hep_deg_3d = (hip_rad_3d*180)/math.pi
print("rad:", hip_rad_3d, "/ abs rad:", abs(hip_rad_3d))
print("deg:", hep_deg_3d, "/ abs_deg:", abs(hep_deg_3d))

# 어깨 기울기 검사
print("\n-------------어깨 기울기-------------")
fframe = 10 # 전면 이미지 or 프레임
val = motion_data[fframe]
l_shoulder = val[11]
r_shoulder = val[14]
shoulder_xz = cal_2d_distance(l_shoulder[0], l_shoulder[2], r_shoulder[0], r_shoulder[2])
shoulder_rad_3d = math.atan2(l_shoulder[1] - r_shoulder[1], shoulder_xz)
shoulder_deg_3d = (shoulder_rad_3d*180)/math.pi
print("rad:", shoulder_rad_3d, "/ abs rad:", abs(shoulder_rad_3d))
print("deg:", shoulder_deg_3d, "/ abs_deg:", abs(shoulder_deg_3d))


---------------거북목---------------
rad: -1.2737014458393237 / abs rad: 1.2737014458393237
deg: -72.97771720630405 / abs_deg: 72.97771720630405

-------------골반 기울기-------------
rad: -0.05884672846882429 / abs rad: 0.05884672846882429
deg: -3.371669179415981 / abs_deg: 3.371669179415981

-------------어깨 기울기-------------
rad: -0.27983190933826185 / abs rad: 0.27983190933826185
deg: -16.03318737816989 / abs_deg: 16.03318737816989


In [43]:
copyed_data = motion_data.copy()
data_len = len(copyed_data)
threshold = int(data_len * 0.2)
print("threshold:", threshold)
print(copyed_data.shape)
middle_data = copyed_data[threshold:data_len-threshold]
print("middle_data")
print(middle_data.shape)
mean_data = np.mean(middle_data, axis=0)
print("mean_data")
print(mean_data.shape)

threshold: 25
(129, 17, 3)
middle_data
(79, 17, 3)
mean_data
(17, 3)


In [44]:
# 거북목 증후군 검사
print("\n---------------거북목---------------")
head = mean_data[10]
neck = mean_data[8]
tutle_xz = cal_2d_distance(head[0], head[2], neck[0], neck[2])
turtle_rad_3dm = math.atan2(head[1] - neck[1], tutle_xz)
turtle_deg_3dm = (turtle_rad_3dm*180)/math.pi
print("rad:", turtle_rad_3dm, "/ abs rad:", abs(turtle_rad_3dm))
print("deg:", turtle_deg_3dm, "/ abs_deg:", abs(turtle_deg_3dm))

# 골반 기울기 검사
print("\n-------------골반 기울기-------------")
l_hip = mean_data[4]
r_hip = mean_data[1]
hip_xz = cal_2d_distance(l_hip[0], l_hip[2], r_hip[0], r_hip[2])
hip_rad_3dm = math.atan2(l_hip[1] - r_hip[1], hip_xz)
hep_deg_3dm = (hip_rad_3dm*180)/math.pi
print("rad:", hip_rad_3dm, "/ abs rad:", abs(hip_rad_3dm))
print("deg:", hep_deg_3dm, "/ abs_deg:", abs(hep_deg_3dm))

# 어깨 기울기 검사
print("\n-------------어깨 기울기-------------")
l_shoulder = mean_data[11]
r_shoulder = mean_data[14]
shoulder_xz = cal_2d_distance(l_shoulder[0], l_shoulder[2], r_shoulder[0], r_shoulder[2])
shoulder_rad_3dm = math.atan2(l_shoulder[1] - r_shoulder[1], shoulder_xz)
shoulder_deg_3dm = (shoulder_rad_3dm*180)/math.pi
print("rad:", shoulder_rad_3dm, "/ abs rad:", abs(shoulder_rad_3dm))
print("deg:", shoulder_deg_3dm, "/ abs_deg:", abs(shoulder_deg_3dm))


---------------거북목---------------
rad: -1.2474560551035576 / abs rad: 1.2474560551035576
deg: -71.4739670854729 / abs_deg: 71.4739670854729

-------------골반 기울기-------------
rad: -0.04866046800162735 / abs rad: 0.04866046800162735
deg: -2.7880394456246385 / abs_deg: 2.7880394456246385

-------------어깨 기울기-------------
rad: -0.26929184342668594 / abs rad: 0.26929184342668594
deg: -15.429286085646885 / abs_deg: 15.429286085646885


In [45]:
def extract_frame(video_path, frame_number=0):
    # 비디오 파일 열기
    cap = cv2.VideoCapture(video_path)

    # 파일이 열렸는지 확인
    if not cap.isOpened():
        print("Error: Could not open video.")
        return None

    # 특정 프레임으로 이동
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        print(f"Error: Could not read frame {frame_number}.")
        return None

    # 비디오 파일 닫기
    cap.release()

    return frame

In [46]:
origin_path = 'AlphaPose/examples/video/unnormal.mp4'
alpha_path = 'AlphaPose/examples/results/AlphaPose_unnormal.mp4'
motion_path = 'MotionBERT/examples/results/motionbert_unnormal.mp4'
mesh_path = 'MotionBERT/examples/results/motionbert_mesh_unnormal.mp4'
path_list = [origin_path, alpha_path, motion_path, mesh_path]

frame_number = len(motion_data) // 2  # middle frame of video
print("frame number:", frame_number)

for i in range(len(path_list)):
    frame = extract_frame(path_list[i], frame_number)

    if frame is not None:
        # 프레임을 이미지 파일로 저장
        cv2.imwrite(f'visualexample/{i}.jpg', frame)
        print(f"Frame extracted and saved as visualexample/{i}.jpg.")

frame number: 64


Frame extracted and saved as visualexample/0.jpg.
Frame extracted and saved as visualexample/1.jpg.
Frame extracted and saved as visualexample/2.jpg.
Frame extracted and saved as visualexample/3.jpg.


In [72]:
def draw_keypoints_and_angle(image_path):
    # 이미지를 읽음
    image = cv2.imread(image_path)

    # 어깨 각도 그리기
    height, width, _ = image.shape
    start_point_horizontal = (0, int(alpha_frame[19]))
    end_point_horizontal = (width, int(alpha_frame[19]))
    cv2.line(image, start_point_horizontal, end_point_horizontal, (0, 0, 255), 2)
    cv2.line(image, (int(alpha_frame[18]), int(alpha_frame[19])), (int(alpha_frame[15]), int(alpha_frame[16])), (0, 0, 255), 2)
    
    angle_text = f"degree: {abs(shoulder_rad*180)/math.pi:.2f}"
    text_position = (int(alpha_frame[18]) + 10, int(alpha_frame[19]) + 25)
    cv2.putText(image, angle_text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # 골반 각도 그리기
    height, width, _ = image.shape
    start_point_horizontal = (0, int(alpha_frame[37]))
    end_point_horizontal = (width, int(alpha_frame[37]))
    cv2.line(image, start_point_horizontal, end_point_horizontal, (0, 0, 255), 2)
    cv2.line(image, (int(alpha_frame[36]), int(alpha_frame[37])), (int(alpha_frame[33]), int(alpha_frame[34])), (0, 0, 255), 2)
    
    angle_text = f"degree: {abs(hip_rad*180)/math.pi:.2f}"
    text_position = (int(alpha_frame[36]) + 10, int(alpha_frame[37]) + 25)
    cv2.putText(image, angle_text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imwrite('visualexample/4.jpg', image)

image_path = f'visualexample/1.jpg'
alpha_frame = alpha_data[frame_number]['keypoints']


draw_keypoints_and_angle(image_path)

In [73]:
def resize_and_concatenate(images):
    # 첫 번째 이미지의 크기를 기준으로 삼기
    base_image = cv2.imread(images[0])
    base_height, base_width = base_image.shape[:2]

    resized_images = []
    
    for image_path in images:
        # 이미지 읽기
        image = cv2.imread(image_path)
        
        # 이미지 리사이즈
        resized_image = cv2.resize(image, (base_width, base_height))
        resized_images.append(resized_image)
    
    # 이미지들을 일렬로 연결
    concatenated_image = np.hstack(resized_images)
    
    return concatenated_image

# 사용 예제
image_paths = [
    'visualexample/0.jpg',
    'visualexample/4.jpg',
    'visualexample/2.jpg',
    'visualexample/3.jpg'
]

result_image = resize_and_concatenate(image_paths)

# 결과 이미지 저장 및 표시
output_path = 'visualexample/5.jpg'
cv2.imwrite(output_path, result_image)
print("Done!")

Done!
